# Gererate test spectro, phot and like_calc pickles

Generate the Python pickles needed to run the unit tests in `test_spectro.py`, `test_phot.py` and `test_like_calc.py`. This notebook should only be run to redefine the cosmology dictionary (i.e. `Cosmology.cosmo_dic`) used for unit tests. 

> ⚠️ WARNING! Changing the values in these pickles make break current tests.

## Set Up

In [ ]:
import copy
import pickle
import numpy as np
import sys
from astropy import constants as const
from os.path import join
from pathlib import Path
from scipy import interpolate

Update Python path to find CLOE.

In [ ]:
sys.path.insert(0, str(Path().resolve().parents[3]))

Import CLOE test routines.

In [ ]:
from cloe.tests.test_tools.mock_objects import update_dict_w_mock
from cloe.tests.test_tools import test_data_handler as tdh

Set path to `test_input`.

In [ ]:
path = join(Path().resolve().parents[1], 'test_input')

## Define cosmology dictionary

Set the base cosmological parameters.

> ⚠️ WARNING! This should be updated to use `cloe.tests.test_input.base_test_cosmo.base_cosmo_dic` at some point.


In [ ]:
mock_cosmo_dic = {
    'c': const.c.to('km/s').value,
    'ombh2': 0.022445, 
    'omch2': 0.121203, 
    'H0': 67.0,
    'tau': 0.07, 
    'mnu': 0.06, 
    'nnu': 3.046,
    'omkh2': 0.0, 
    'omnuh2': 0.0, 
    'ns': 0.96,
    'w': -1.0, 
    'sigma8_0': 0.816,
    'As': 2.115e-9, 
}

Load test data from `cloe/tests/test_input`.

In [ ]:
cmov_file = tdh.load_test_npy('ComDist-LCDM-Lin-zNLA.dat', path)
Hz_file = tdh.load_test_npy('Hz.dat', path)
f_sig_8_arr = tdh.load_test_npy('f_sig_8_arr.npy', path)
sig_8_arr = tdh.load_test_npy('sig_8_arr.npy', path)
f_z_arr = tdh.load_test_npy('f_z_arr.npy', path)
pdd = tdh.load_test_npy('pdd.npy', path)
pdi = tdh.load_test_npy('pdi.npy', path)
pgd = tdh.load_test_npy('pgd.npy', path)
pgg = tdh.load_test_npy('pgg.npy', path)
pgi_phot = tdh.load_test_npy('pgi_phot.npy', path)
pgi_spectro = tdh.load_test_npy('pgi_spectro.npy', path)
pii = tdh.load_test_npy('pii.npy', path)
fid_H_arr = tdh.load_test_npy('spectro_fid_HZ.npy', path)
fid_d_A_arr = tdh.load_test_npy('spectro_fid_d_A.npy', path)

Interpolate loaded data.

In [ ]:
zs_r = cmov_file[:, 0]
rs = cmov_file[:, 1]
ang_dists = rs / (1.0 + zs_r)

rz_interp = interpolate.InterpolatedUnivariateSpline(x=zs_r, y=rs, ext=0)
zr_interp = interpolate.InterpolatedUnivariateSpline(x=rs, y=zs_r, ext=0)
dz_interp = interpolate.InterpolatedUnivariateSpline(x=zs_r, y=ang_dists, ext=0)

zs_H = Hz_file[:, 0]
Hs = Hz_file[:, 1]
Hs_mpc = Hz_file[:, 1] / const.c.to('km/s').value

Hz_interp = interpolate.InterpolatedUnivariateSpline(x=zs_H, y=Hs, ext=0)
Hmpc_interp = interpolate.InterpolatedUnivariateSpline(x=zs_H, y=Hs_mpc, ext=0)

sig_8_interp = interpolate.InterpolatedUnivariateSpline(
    x=np.linspace(0.0, 5.0, 50),
    y=sig_8_arr[::-1], 
    ext=0
)
f_sig_8_interp = interpolate.InterpolatedUnivariateSpline(
    x=np.linspace(0.0, 5.0, 50),
    y=f_sig_8_arr[::-1], 
    ext=0
)
f_z_interp = interpolate.InterpolatedUnivariateSpline(
    x=np.linspace(0.0, 5.0, 50),
    y=f_z_arr[::-1], 
    ext=0
)
luminosity_ratio_z_interp = interpolate.InterpolatedUnivariateSpline(
    x=np.linspace(0.0, 2.1, 100),
    y=np.linspace(0.01, 1.7, 100), 
    ext=0
)


zs_base = np.linspace(0.0, 4.0, 100)
ks_base = np.logspace(-3.0, 1.0, 100)

fid_H_interp = interpolate.InterpolatedUnivariateSpline(x=zs_H, y=fid_H_arr, ext=0)
fid_dA_interp = interpolate.InterpolatedUnivariateSpline(x=zs_H, y=fid_d_A_arr, ext=0)

Update the cosmology dictionary with interpolations and precomputed values.

In [ ]:
mock_cosmo_dic.update({
    'sigma8_z_func': sig_8_interp,
    'fsigma8_z_func': f_sig_8_interp,
    'luminosity_ratio': luminosity_ratio_z_interp,
    'f_z': f_z_interp,
    'r_z_func': rz_interp, 
    'f_K_z_func': rz_interp,
    'z_r_func': zr_interp,
    'd_z_func': dz_interp,
    'H_z_func_Mpc': Hmpc_interp,
    'H_z_func': Hz_interp,
    'z_win': zs_base,
    'k_win': ks_base,
})

mock_cosmo_dic['H0_Mpc'] = mock_cosmo_dic['H0'] / const.c.to('km/s').value
mock_cosmo_dic['Omb'] = mock_cosmo_dic['ombh2'] / (mock_cosmo_dic['H0'] / 100.0) ** 2.0
mock_cosmo_dic['Omc'] = mock_cosmo_dic['omch2'] / (mock_cosmo_dic['H0'] / 100.0) ** 2.0
mock_cosmo_dic['Omnu'] = mock_cosmo_dic['omnuh2'] / (mock_cosmo_dic['H0'] / 100.0) ** 2.0
mock_cosmo_dic['Omm'] = (
    mock_cosmo_dic['Omnu'] + mock_cosmo_dic['Omc'] + mock_cosmo_dic['Omb']
)
mock_cosmo_dic['Omk'] = mock_cosmo_dic['omkh2'] / (mock_cosmo_dic['H0'] / 100.0) ** 2.0
mock_cosmo_dic['fid_d_z_func'] = fid_dA_interp
mock_cosmo_dic['fid_H_z_func'] = fid_H_interp

In [ ]:
mock_cosmo_dic['obs_selection'] = {
        'WL': {'WL': True, 'GCphot': True, 'GCspectro': False},
        'GCphot': {'GCphot': True, 'GCspectro': False},
        'GCspectro': {'GCspectro': True},
        'add_phot_RSD': False
    }

## Set bias model

In [ ]:
mock_cosmo_dic['bias_model'] = 1
mock_cosmo_dic['magbias_model'] = 2

## Set parameters for `test_spectro`

Set nuisance parameters.

In [ ]:
nuisance_spectro = {
    'b1_photo': 1.0997727037892875,
    'b2_photo': 1.220245876862528,
    'b3_photo': 1.2723993083933989,
    'b4_photo': 1.316624471897739,
    'b5_photo': 1.35812370570578,
    'b6_photo': 1.3998214171814918,
    'b7_photo': 1.4446452851824907,
    'b8_photo': 1.4964959071110084,
    'b9_photo': 1.5652475842498528,
    'b10_photo': 1.7429859437184225,
    'b1_spectro_bin1': 1.4614804,
    'b1_spectro_bin2': 1.6060949,
    'b1_spectro_bin3': 1.7464790,
    'b1_spectro_bin4': 1.8988660,
    'aia': 1.72,
    'nia': -0.41,
    'bia': 0.0,
    'pivot_redshift': 0.,
    'dz_1_GCphot': 0.0, 
    'dz_1_WL': 0.0,
    'dz_2_GCphot': 0.0, 
    'dz_2_WL': 0.0,
    'dz_3_GCphot': 0.0, 
    'dz_3_WL': 0.0,
    'dz_4_GCphot': 0.0, 
    'dz_4_WL': 0.0,
    'dz_5_GCphot': 0.0, 
    'dz_5_WL': 0.0,
    'dz_6_GCphot': 0.0, 
    'dz_6_WL': 0.0,
    'dz_7_GCphot': 0.0, 
    'dz_7_WL': 0.0,
    'dz_8_GCphot': 0.0, 
    'dz_8_WL': 0.0,
    'dz_9_GCphot': 0.0, 
    'dz_9_WL': 0.0,
    'dz_10_GCphot': 0.0, 
    'dz_10_WL': 0.0,
    'f_out': 0.0,
    'f_out_1': 0.0,
    'f_out_2': 0.0,
    'f_out_3': 0.0,
    'f_out_4': 0.0
}

for index in range(10):
    nuisance_spectro[f'multiplicative_bias_{index + 1}'] = 0
    nuisance_spectro[f'magnification_bias_{index + 1}'] = 0

Copy cosmology dictionary, set `test_spectro`-specific parameters, assign mock test objects, and save the dictionary to a pickle file in `cloe/tests/test_input`.

In [ ]:
test_spectro_dic = copy.deepcopy(mock_cosmo_dic)

test_spectro_dic['NL_flag_phot_matter'] = 1
test_spectro_dic['NL_flag_spectro'] = 1
test_spectro_dic['bias_model'] = 1
test_spectro_dic['f_out_z_dep'] = False
test_spectro_dic['nuisance_parameters'] = nuisance_spectro
test_spectro_dic['Pdd_phot'] = interpolate.interp2d(zs_base, ks_base, pdd.T, fill_value=0.0)
test_spectro_dic['Pgg_phot'] = interpolate.interp2d(zs_base, ks_base, pgg.T, fill_value=0.0)
test_spectro_dic['Pgdelta_phot'] = interpolate.interp2d(zs_base, ks_base, pgd.T, fill_value=0.0)
test_spectro_dic['Pii'] = interpolate.interp2d(zs_base, ks_base, pii.T, fill_value=0.0)
test_spectro_dic['Pdeltai'] = interpolate.interp2d(zs_base, ks_base, pdi.T, fill_value=0.0)
test_spectro_dic['Pgi_phot'] = interpolate.interp2d(zs_base, ks_base, pgi_phot.T, fill_value=0.0)
test_spectro_dic['Pgi_spectro'] = interpolate.interp2d(zs_base, ks_base, pgi_spectro.T, fill_value=0.0)

test_spectro_dic = update_dict_w_mock(test_spectro_dic)

tdh.save_test_pickle('spectro_test_dic.pickle', test_spectro_dic, path)

## Set parameters for `test_phot`

Set nuisance parameters.

In [ ]:
nuisance_phot = {}
for index in range(10):
    nuisance_phot[f'dz_{index + 1}_GCphot'] = 0.0
    nuisance_phot[f'dz_{index + 1}_WL'] = 0.0
    nuisance_phot[f'b{index + 1}_photo'] = 1.0
    nuisance_phot[f'multiplicative_bias_{index + 1}'] = 0.0
    nuisance_phot[f'magnification_bias_{index + 1}'] = 0.0

Copy cosmology dictionary, set `test_phot`-specific parameters, assign mock test objects, and save the dictionary to a pickle file in `cloe/tests/test_input`.

In [ ]:
test_phot_dic = copy.deepcopy(mock_cosmo_dic)
test_phot_dic['redshift_bins_means_phot'] = [0.2095, 0.489, 0.619, 0.7335, 0.8445, 0.9595, 1.087, 1.2395, 1.45, 2.038]
test_phot_dic['nuisance_parameters'] = nuisance_phot
test_phot_dic['Pmm_phot'] = interpolate.RectBivariateSpline(zs_base, ks_base, pdd, kx=1, ky=1)
test_phot_dic['Pgg_phot'] = interpolate.RectBivariateSpline(zs_base, ks_base, pgg, kx=1, ky=1)
test_phot_dic['Pgdelta_phot'] = interpolate.RectBivariateSpline(zs_base, ks_base, pgd, kx=1, ky=1)
test_phot_dic['Pii'] = interpolate.RectBivariateSpline(zs_base, ks_base, pii, kx=1, ky=1)
test_phot_dic['Pdeltai'] = interpolate.RectBivariateSpline(zs_base, ks_base, pdi, kx=1, ky=1)
test_phot_dic['Pgi_phot'] = interpolate.RectBivariateSpline(zs_base, ks_base, pgi_phot, kx=1, ky=1)
test_phot_dic['Flat_nz'] = interpolate.InterpolatedUnivariateSpline(
    np.linspace(0.0, 4.6, 20), 
    np.ones(20), 
    ext=2,
)

from cloe.cosmo.cosmology import Cosmology
cosmo = Cosmology()
cosmo.cosmo_dic = test_phot_dic
cosmo.create_phot_galbias(model = 1)
test_phot_dic['b_inter'] = cosmo.cosmo_dic['b_inter']

test_phot_dic = update_dict_w_mock(test_phot_dic)
    
tdh.save_test_pickle('phot_test_dic.pickle', test_phot_dic, path)

## Set parameters for `test_like_calc`

Copy cosmology dictionary, set `test_like_calc`-specific parameters, assign mock test objects, and save the dictionary to a pickle file in `cloe/tests/test_input`.

In [ ]:
test_like_calc_dic = copy.deepcopy(mock_cosmo_dic)

test_like_calc_dic['f_out_z_dep'] = False
test_like_calc_dic['nuisance_parameters'] = nuisance_spectro
test_like_calc_dic['Pdd_phot'] = interpolate.interp2d(zs_base, ks_base, pdd.T, fill_value=0.0)
test_like_calc_dic['Pmm_phot'] = interpolate.RectBivariateSpline(zs_base, ks_base, pdd, kx=1, ky=1)
test_like_calc_dic['Pgg_phot'] = interpolate.RectBivariateSpline(zs_base, ks_base, pgg, kx=1, ky=1)
test_like_calc_dic['Pgdelta_phot'] = interpolate.RectBivariateSpline(zs_base, ks_base, pgd, kx=1, ky=1)
test_like_calc_dic['Pii'] = interpolate.RectBivariateSpline(zs_base, ks_base, pii, kx=1, ky=1)
test_like_calc_dic['Pdeltai'] = interpolate.RectBivariateSpline(zs_base, ks_base, pdi, kx=1, ky=1)
test_like_calc_dic['Pgi_phot'] = interpolate.RectBivariateSpline(zs_base, ks_base, pgi_phot, kx=1, ky=1)
test_like_calc_dic['Pgi_spectro'] = interpolate.RectBivariateSpline(zs_base, ks_base, pgi_spectro, kx=1, ky=1)

test_like_calc_dic = update_dict_w_mock(test_like_calc_dic)
    
tdh.save_test_pickle('like_calc_test_dic.pickle', test_like_calc_dic, path)